In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
import IPython
from sklearn.preprocessing import StandardScaler

Reading in Data

In [2]:
#Katie

# Read In csvs
plays =  pd.read_csv("plays.csv")
players = pd.read_csv("players.csv")
week_1 = pd.read_csv("tracking_week_1.csv")
week_2 = pd.read_csv("tracking_week_2.csv")
week_3 = pd.read_csv("tracking_week_3.csv")
week_4 = pd.read_csv("tracking_week_4.csv")
week_5 = pd.read_csv("tracking_week_5.csv")
week_6 = pd.read_csv("tracking_week_6.csv")
week_7 = pd.read_csv("tracking_week_7.csv")
week_8 = pd.read_csv("tracking_week_8.csv")
week_9 = pd.read_csv("tracking_week_9.csv")
tackles = pd.read_csv("tackles.csv")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

(12187398, 18)

In [ ]:
"""Emily's reading in files"""
data_dir = os.getcwd() + '/data/nfl-big-data-bowl-2024/'
players = pd.read_csv(data_dir + "players.csv")
week_1 = pd.read_csv(data_dir + "tracking_week_1.csv")
week_2 = pd.read_csv(data_dir + "tracking_week_2.csv")
week_3 = pd.read_csv(data_dir + "tracking_week_3.csv")
week_4 = pd.read_csv(data_dir + "tracking_week_4.csv")
week_5 = pd.read_csv(data_dir + "tracking_week_5.csv")
week_6 = pd.read_csv(data_dir + "tracking_week_6.csv")
week_7 = pd.read_csv(data_dir + "tracking_week_7.csv")
week_8 = pd.read_csv(data_dir + "tracking_week_8.csv")
week_9 = pd.read_csv(data_dir + "tracking_week_9.csv")
plays = pd.read_csv(data_dir + "plays.csv")
tackles = pd.read_csv(data_dir + "tackles.csv")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

# join player positioning information onto a week's worth of tracking data 
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')

In [ ]:
#Matt
def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
plays = load_dataset("plays")
players = load_dataset("players")
week_1 = load_dataset("tracking_week_1")
week_2 = load_dataset("tracking_week_2")
week_3 = load_dataset("tracking_week_3")
week_4 = load_dataset("tracking_week_4")
week_5 = load_dataset("tracking_week_5")
week_6 = load_dataset("tracking_week_6")
week_7 = load_dataset("tracking_week_7")
week_8 = load_dataset("tracking_week_8")
week_9 = load_dataset("tracking_week_9")
tackles = load_dataset("tackles")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape


Creating df with input and output data

In [3]:
# get distance from ball
ball = week.groupby(['gameId','playId', 'frameId']).apply(lambda g: g[g['club'] == 'football'])[['x','y']]
week_and_ball = week.merge(ball, on = ['gameId','playId', 'frameId'], how = 'left', suffixes=('','_ball') )
week_and_ball['distance_to_ball'] = np.sqrt((week_and_ball["x"] - week_and_ball["x_ball"]) ** 2 + (week_and_ball["y"] - week_and_ball["y_ball"]) ** 2)
print('1')


#sort by position and jerseyNumber
week_and_ball = week_and_ball.sort_values(['gameId','playId', 'frameId', 'club', 'position', 'jerseyNumber'])

#merge with tackles
input = week_and_ball.copy()
input = input.merge(tackles[['gameId', 'playId', 'nflId', 'tackle', 'assist']], on = ['gameId', 'playId', 'nflId'], how = 'left')
input['tackle'] = input['tackle'].fillna(0)
input['assist'] = input['assist'].fillna(0)

print('2')


#get order of players (0-10)
input['number'] = input.groupby(['gameId','playId', 'frameId', 'club']).cumcount()
input = input[input['club'] != 'football']
# check if needed
input = input.sort_values(by = ['gameId', 'playId', 'frameId', 'position', 'jerseyNumber'])
input = input.reset_index(drop = True)

# merge with plays
input_transform = input.copy()
#check
input_transform = input_transform.reset_index(drop = True)
plays = plays[['gameId', 'playId', 'possessionTeam', 'defensiveTeam']]
all_players = input.merge(plays, on = ['gameId', 'playId'])

print('3')

# offense
offense = all_players[all_players['possessionTeam'] == all_players['club']]
offense = offense.rename(columns={c: 'o_'+c for c in offense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

o_table = pd.pivot_table(offense, values=['o_x', 'o_y', 'o_s', 'o_a', 'o_dis', 'o_o', 'o_dir', 'o_distance_to_ball'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
o_table.columns = o_table.columns.map('{0[0]}_{0[1]}'.format) 
o_table = o_table.reset_index(['gameId','playId','frameId'])

# defense - includes output data
defense = all_players[all_players['defensiveTeam'] == all_players['club']]
defense = defense.rename(columns={c: 'd_'+c for c in defense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

d_table = pd.pivot_table(defense, values=['d_x', 'd_y', 'd_s', 'd_a', 'd_dis', 'd_o', 'd_dir', 'd_distance_to_ball', 'tackle', 'assist'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
d_table.columns = d_table.columns.map('{0[0]}_{0[1]}'.format) 
d_table = d_table.reset_index(['gameId','playId','frameId'])

# combinate offense and defense
input_output_df = o_table.merge(d_table, on = ['gameId', 'playId', 'frameId'], how = 'right')

# calculate tackle credit 
tackle_cols = ['tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10']
assist_cols = ['assist_0', 'assist_1', 'assist_2', 'assist_3', 'assist_4', 'assist_5', 'assist_6', 'assist_7', 'assist_8', 'assist_9', 'assist_10']
tackle_assist_cols = tackle_cols + assist_cols

input_output_df['total_tacklers'] = input_output_df[tackle_assist_cols].sum(axis=1)
input_output_df['tackle_11'] = np.where(input_output_df['total_tacklers'] == 0 , 1, 0)
input_output_df.loc[:, tackle_cols] = input_output_df.loc[:, tackle_cols].div(input_output_df['total_tacklers'], axis=0).fillna(0)

# drop unnecessary columns
input_output_df.drop(assist_cols, axis=1, inplace=True)
input_output_df.drop('total_tacklers', axis=1, inplace=True)

1
2
3


In [4]:
# List of columns representing the distance of each tackler from the ball
distance_columns = [
    'd_distance_to_ball_0', 'd_distance_to_ball_1', 'd_distance_to_ball_2',
    'd_distance_to_ball_3', 'd_distance_to_ball_4', 'd_distance_to_ball_5',
    'd_distance_to_ball_6', 'd_distance_to_ball_7', 'd_distance_to_ball_8',
    'd_distance_to_ball_9', 'd_distance_to_ball_10'
]

# Identify the tackler closest to the ball for each frame
#This function is used to find the column name with the minimum value for each row, which indicates the closest tackler for that frame
input_output_df['closest_tackler'] = input_output_df[distance_columns].idxmin(axis=1)

# Extract the tackler number from the column name and create a new column
input_output_df['closest_tackler'] = input_output_df['closest_tackler'].str.extract('(\d+)').astype(int)


In [12]:
#reorder so it is not at the end 

# Identify the index of 'tackle_0' column
tackle_0_index = input_output_df.columns.get_loc("o_a_0")

# Create a new order for columns
# Get all columns up to 'tackle_0', add 'closest_tackler', and then the rest
new_column_order = input_output_df.columns[:tackle_0_index].tolist() + ['closest_tackler'] + input_output_df.columns[tackle_0_index:-1].tolist()

# Reorder the DataFrame
input_output_df = input_output_df[new_column_order]


In [13]:
input_output_df

,gameId,playId,frameId,closest_tackler,o_a_0,o_a_1,o_a_2,o_a_3,o_a_4,o_a_5,...,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10
0,2022090800,56,1,8,1.97,2.46,1.15,1.00,0.33,2.47,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022090800,56,2,7,1.82,2.53,0.61,1.22,0.66,2.59,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022090800,56,3,1,1.60,2.73,0.49,1.24,0.92,2.56,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022090800,56,4,1,1.26,2.74,0.89,1.03,1.37,2.40,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022090800,56,5,1,0.99,2.94,1.24,0.76,1.63,2.50,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529874,2022110700,3787,40,3,1.30,0.94,0.41,4.04,1.30,1.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529875,2022110700,3787,41,6,1.54,0.91,0.28,4.24,1.46,1.24,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529876,2022110700,3787,42,6,1.58,0.94,0.27,4.13,1.50,1.25,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529877,2022110700,3787,43,6,1.60,1.01,0.48,3.65,1.61,1.32,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# cell I used to confirm my new closest_tackler was correct

with pd.option_context('display.max_columns', None):  
    print(input_output_df.iloc[0:3])

       gameId  playId  frameId  closest_tackler  o_a_0  o_a_1  o_a_2  o_a_3  \
0  2022090800      56        1                8   1.97   2.46   1.15   1.00   
1  2022090800      56        2                7   1.82   2.53   0.61   1.22   
2  2022090800      56        3                1   1.60   2.73   0.49   1.24   

   o_a_4  o_a_5  o_a_6  o_a_7  o_a_8  o_a_9  o_a_10  o_dir_0  o_dir_1  \
0   0.33   2.47   2.57   4.49   6.34   0.79    0.90    30.29   115.97   
1   0.66   2.59   2.75   4.09   6.53   0.81    1.27    10.32   121.60   
2   0.92   2.56   2.90   3.72   6.70   0.78    2.48     0.28   128.46   

   o_dir_2  o_dir_3  o_dir_4  o_dir_5  o_dir_6  o_dir_7  o_dir_8  o_dir_9  \
0   147.90   301.11   238.13    65.25   117.85   220.71   233.63   260.37   
1   148.53   287.34   237.69    61.23   122.35   216.36   224.03   259.74   
2   147.05   187.42   236.36    54.47   129.79   212.28   213.17   259.50   

   o_dir_10  o_dis_0  o_dis_1  o_dis_2  o_dis_3  o_dis_4  o_dis_5  o_dis_6  \
0  

In [ ]:
input_output_df.to_csv('input_output_df_base.csv')

In [10]:
for i in input_output_df.columns:
    print(i)

gameId
playId
frameId
o_a_0
o_a_1
o_a_2
o_a_3
o_a_4
o_a_5
o_a_6
o_a_7
o_a_8
o_a_9
o_a_10
o_dir_0
o_dir_1
o_dir_2
o_dir_3
o_dir_4
o_dir_5
o_dir_6
o_dir_7
o_dir_8
o_dir_9
o_dir_10
o_dis_0
o_dis_1
o_dis_2
o_dis_3
o_dis_4
o_dis_5
o_dis_6
o_dis_7
o_dis_8
o_dis_9
o_dis_10
o_distance_to_ball_0
o_distance_to_ball_1
o_distance_to_ball_2
o_distance_to_ball_3
o_distance_to_ball_4
o_distance_to_ball_5
o_distance_to_ball_6
o_distance_to_ball_7
o_distance_to_ball_8
o_distance_to_ball_9
o_distance_to_ball_10
o_o_0
o_o_1
o_o_2
o_o_3
o_o_4
o_o_5
o_o_6
o_o_7
o_o_8
o_o_9
o_o_10
o_s_0
o_s_1
o_s_2
o_s_3
o_s_4
o_s_5
o_s_6
o_s_7
o_s_8
o_s_9
o_s_10
o_x_0
o_x_1
o_x_2
o_x_3
o_x_4
o_x_5
o_x_6
o_x_7
o_x_8
o_x_9
o_x_10
o_y_0
o_y_1
o_y_2
o_y_3
o_y_4
o_y_5
o_y_6
o_y_7
o_y_8
o_y_9
o_y_10
d_a_0
d_a_1
d_a_2
d_a_3
d_a_4
d_a_5
d_a_6
d_a_7
d_a_8
d_a_9
d_a_10
d_dir_0
d_dir_1
d_dir_2
d_dir_3
d_dir_4
d_dir_5
d_dir_6
d_dir_7
d_dir_8
d_dir_9
d_dir_10
d_dis_0
d_dis_1
d_dis_2
d_dis_3
d_dis_4
d_dis_5
d_dis_6
d_dis_7
d_dis_8
d_dis

In [11]:
#Katie's work checking how many times each tackler is used

tackle_cols = ['tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11']

# Initialize dictionaries for counting distinct occurrences
distinct_counts_1 = {col: 0 for col in tackle_cols}  # For full tackles
distinct_counts_fractional = {col: 0 for col in tackle_cols}  # For fractional tackles

# Sets to keep track of processed game and play IDs for each player
processed_1 = {col: set() for col in tackle_cols}
processed_fractional = {col: set() for col in tackle_cols}

for _, row in input_output_df.iterrows():
    game_play_key = (row['gameId'], row['playId'])
    # Handle full tackles (1)
    for col in tackle_cols:
        if row[col] == 1 and game_play_key not in processed_1[col]:
            distinct_counts_1[col] += 1
            processed_1[col].add(game_play_key)

    # Handle fractional tackles (greater than 0 but less than 1..accounts for .5 and .33 tackles)
    involved_fractional = [col for col in tackle_cols if 0 < row[col] < 1]
    for col in involved_fractional:
        if game_play_key not in processed_fractional[col]:
            distinct_counts_fractional[col] += 1
            processed_fractional[col].add(game_play_key)
            print 

distinct_counts_df_1 = pd.DataFrame(list(distinct_counts_1.items()), columns=['Column', 'Distinct Count 1s']).sort_values('Column')
distinct_counts_df_fractional = pd.DataFrame(list(distinct_counts_fractional.items()), columns=['Column', 'Distinct Count Fractional Tackles']).sort_values('Column')

print(distinct_counts_df_1)
print(distinct_counts_df_fractional)



       Column  Distinct Count 1s
0    tackle_0                839
1    tackle_1                848
10  tackle_10                922
11  tackle_11                564
2    tackle_2                791
3    tackle_3                484
4    tackle_4                529
5    tackle_5                719
6    tackle_6                852
7    tackle_7                864
8    tackle_8                759
9    tackle_9                825
       Column  Distinct Count Fractional Tackles
0    tackle_0                                 97
1    tackle_1                                 87
10  tackle_10                                135
11  tackle_11                                  0
2    tackle_2                                125
3    tackle_3                                133
4    tackle_4                                107
5    tackle_5                                133
6    tackle_6                                200
7    tackle_7                                183
8    tackle_8                   

In [17]:
#checking only insatces of 1/3 tackles 

one_third_tackles = input_output_df[input_output_df[tackle_cols].apply(lambda x: (x == 1/3).any(), axis=1)]

for _, row in one_third_tackles.iterrows():
    print("Game ID:", row['gameId'], "| Play ID:", row['playId'])
    print(row[tackle_cols])  # Print all tackle columns for this row
    print("\n") 


KeyError: "['tackle_11'] not in index"

In [ ]:


just_input = input_output_df.iloc[:, 3:-12]
just_input

In [ ]:
just_output = input_output_df.iloc[:, -12:]
just_output

In [ ]:
scaler = StandardScaler()
normalized_input = pd.DataFrame(scaler.fit_transform(just_input), columns=just_input.columns)
normalized_input

In [ ]:
normalized_input_output_df = pd.concat([input_output_df[['gameId', 'playId', 'frameId']], normalized_input, just_output], axis = 1)

In [ ]:
def last_5_converter(df):
  final_input = pd.DataFrame(columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
  for idx, row in df.iterrows():
    if row['frameId'] < 5:
      current = df.iloc[idx, 4:-12].tolist()
      previous_frames = [np.zeros(17) for _ in range(4)]
      #print("FrameID",row['frameId'])
      # Use a loop to assign the lists
      for i in range(1, int(row['frameId'])):
          #print(3-i)
          previous_frames[4-i] = df.iloc[idx - i, 4:-12].tolist()

      new_row = [previous_frames[0], previous_frames[1], previous_frames[2], previous_frames[3], current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
      final_input.loc[len(final_input.index)] = new_row 
    else:
      current = df.iloc[idx, 4:-12].tolist()
      one = df.iloc[idx - 1, 4:-12].tolist()
      two = df.iloc[idx - 2, 4:-12].tolist()
      three = df.iloc[idx - 3, 4:-12].tolist()
      four = df.iloc[idx - 4, 4:-12].tolist()
      new_row = [four, three, two, one, current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
      # Assuming final_input.columns contains the correct column names
      new_row_series = pd.Series(new_row, index=final_input.columns)
      # Concatenate along axis=1
      final_input.loc[len(final_input.index)] = new_row 
      #final_input = pd.concat([final_input,pd.Series(new_row, index=final_input.columns)], axis = 1, ignore_index=True)
  return final_input

In [ ]:
def last_5_converter(df):
    rows = []
    for idx, row in df.iterrows():
        current = df.iloc[idx, 4:-12].tolist()
        tackles = [df.at[idx, f'tackle_{i}'] for i in range(12)]
        if row['frameId'] < 5:
            previous_frames = [np.zeros(17) for _ in range(4)]
            for i in range(1, int(row['frameId'])):
                previous_frames[4-i] = df.iloc[idx - i, 4:-12].tolist()
        else:
            previous_frames = [df.iloc[idx - i, 4:-12].tolist() for i in range(1, 5)]
        new_row = previous_frames + [current] + tackles
        rows.append(new_row)
    final_input = pd.DataFrame(rows, columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
    return final_input

In [ ]:
model_input_no_id = last_5_converter(normalized_input_output_df[:50000])

In [ ]:
len(model_input_no_id)

In [ ]:
model_input_no_id